<a href="https://colab.research.google.com/github/Curiousss/EVA/blob/master/Phase2/Session9/P2Session9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  
  INITIALIZATION  We initialize the Experience Replay Memory, with a size of 20000. We will populate it with each new transition


In [0]:
import os
import time
import random 
import numpy as np 
import matplotlib.pyplot as plt 
import pybullet envs 
import gym 
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
from gym import wrappers
from torch.autograd import Variable 
from collections import deque


We initialize the Experience Replay Memory with a size of `max_size` 1e6.
Then we populate it with new transitions.
Allow the Agent to run randomly and fill up the Replay Buffer `addTransition`. 
We need the Replay Buffer/Memory ready before training the Critic.
We run full episodes with the first 10,000 actions played randomly, and then with actions played by the Actor Model. This is required to fill up the Replay Memory. 


The replay buffer, `storage` is an array and the `ptr` is moved from postion 0 to end as and when the trasitions are added, after reaching the end that is `max_size`, the pointer resets to 0 and the new transitions are over-written and this continues.

`sample` samples a `batch_size` of data randomly from the `storage` replay buffer. the length of `storage` maybe less than or equal to `max_size` while `storage` replay buffer is still getting updated.
`batch_dones` is a `done` switch for each batch to check if the episode is done completely or not.

Make any array each for `batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones`from element of the tuple (`state, next_state, action, reward, done`) which the tranistion from each entry in the `storage`. There will no duplicate copies of these entries but multiple references to the memory location is used. Now return these new arrays.

In [0]:
class ReplayBuffer(object):
  # Init is there for all classes to initialize an object
  # Self is pointer to the object of the class which is initialized
  def __init_(self, max_size = 1e6):
    self.storage =[] 
    self.max_size = max_size 
    self.ptr = 0

def add(self, transition):
  if len(self.storage) == self.max_size:
    self.storage[int(self.ptr)] = transition 
    self.ptr = (self.ptr + 1) % self.max_size
  else:
    self.storage.append(transition)

def sample(self, batch_size):
  ind = np.random.randint (e, len(self.storage), batch_size) 
  batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = [], [], [], [], [] 
  
  for i in ind:
    state, next_state, action, reward, done = self.storage[i] 
    batch_states.append(np.array(state, copy = False)) 
    batch_next_states.append(np.array(next_state, copy = False)) 
    batch_actions.append(np.array(action, copy - False)) 
    batch_rewards.append(np.array(reward, copy - False)) 
    batch_dones.append(np.array(done, copy - False) 
  return np. array (batch_states), np.array(batch_next_states), np.array(batch_actions), np.array(batch_rewards).reshape(-1, 1), \
          np.array(batch_dones).reshape(-1, 1)

We build TWO kinds of actor models. One called the Actor Model and another called the Actor Target.

Build one DNN is defined to be used for both the Actor model and the Actor Target.
The Actor Model is trained using Back Propagation while training on Experience Replay buffer.
The Actor Target is trained using the updates from Actor Model through Polyak Averaging, stabilization algorithm. Using Polyak averaging, our targets are not as reactive to changes as models. Targets are delayed in their training as well, which means we allow our models to stabilize first before we update targets. 

So they get similar to the model only if the model is consistent.   
Actor Model and Actor Target models have exactly the same DNN definitions.



`state_dims` is the state parameters defining the number of variables in a state. These variables define the state in the environment. Eg. Where is the arm of the  robot

`action_dim` is the number of actions that can be taken. Value of each action (eg. Right Left) is a floating point number representing the quantity of a particular action(Eg. 5 degree to the left).

`max_action` is the limit of this quantity of an action (Eg. 90 degrees)

The `forward` function outputs `x` the actions, which is scaled to -1 to 1 using tanh activation and multipled by `max_action` to get the exact quantity of each action.

In [0]:
class Actor (nn. Module):
  def __init__(self, state_dims, action_dim, max_action):
    #Max action is to clip in case we added too much noise
    super(Actor, self).__init__# activate the inheritance 
    self.layer_1 = nn.Linear(state_dims, 400) 
    self.layer_2 = nn.Linear(400, 300) 
    self.layer_3 = nn.Linear(300, action_dim) 
    self.max_action = max_action

def forward(self,x):
  x = F.relu(self.layer_1(x)) 
  x = F.relu(self.layer_2(x)) 
  x = self.max_action * torch.tanh(self.layer_3(x))
  return x

STEP 3 We define 1 DNN for Critic 1 and 2. 
Critic 1 and 2 are just different set of layers in a neural network. `Forward` function returns 2 outputs from 2 different layers from critic 1 and 2. 
We Build 2 DNNs, for the two Critic models and two for the two Critic Targets

We do not have `max_action` in Critic since action is coming from Actor DNN where it is taken care.

Critic takes state `x` and action `u` and concatenates it(vertically) before sending it to the DNN layers

The Q1 function is a separate forward function that does not back propagate. It is used to train the Actor using the layers of Critic 1. It does not matter whether critic 1 or 2 is used here in the long run.


In [0]:
class Critic(nn.Module):
  def __init__(self, state_dims, action_dim):
    #max_action is to clip in case we need added too much noise
    super(Critic, sell).__init__() #actvate the inheritence
    #First Critic Network
    self.layer_1 = nn.Linear(state_dims + action_dim, 400) 
    self.layer_2 = nn.Linear(400, 300) 
    self.layer_3 = nn.Linear (300, action_dim) 
    #Second Critic Network 
    self.layer_4 = nn.Linear(state_dims + action_dim, 400) 
    self.layer_5 = nn.Linear(400, 300) 
    self.layer_6 = nn.Linear(300, action_dim)

def forward(self, x, u): 
  # x is state, u is action 
  xu = torch.cat([x, u], 1) # 1 for vertical concatenation, 0 for Horizontal
  #forward propagation on first Critic
  x1 = F.relu(self.layer_1(xu)) 
  x1 = F.relu(self.layer_2(x1)) 
  x1 = self.layer_3(x1) 
  # forward propagation on second Critic 
  x2 = F.relu(self.layer_4(xu)) 
  x2 = F.relu(self.layer_5(x2))
  x2 = self.layer_6(x2)
  return x1, x2

def Q1(self, x, u): 
  #x state, u- action This is used for updating the Q values 
  xu = torch.cat([x, u], 1) # 1 for vertical concatenation, 0 for horizontal
  x1 = F.relu(self.layer_1(xu))
  x1 = F.relu(self.layer_2(x1)
  x1 = self.layer_3(x1)
  return x1

Training process. Create a T3D class, initialize variables and get ready for step 4

In [0]:
# Select CPU or GPU
device = torch.device('cuda' if torch.cuda.is avallable() else cpu)

# Building the whole Training Process into a class
class T3D(object):
  def __init__(self, state_dims, action_dim, max_action):
    # making sure out T3D can work with any environment
    self.actor = Actor(state_dias, action_din, max_action).to(device) #GD
    self.actor_target = Actor(state_dims, action_dim, max_action).to(device) #Polyak Averaging
    self.actor_target.load_state_dict(self.actor.state_dict)
    #Initializing with model weights to keep them same
    self.actor_optimizer = torch.optim.Adam(self.actor parameters()
    
    self.critic = Critic(state_dims, action_dim).to(device) #GD 
    self.critic_target = critic(state_dims, action_dim).to(device) #Polyak Averaging 
    self.critic_target.load_state_dict(self.critic.state_dict) 
    #Initializing with model weights to keep the same 
    self.critic_optimizer = torch.optim.Adam(self.critic.parameters()) 
    self.max_action = max_action

def select_action(self, state):
  state = torch.Tensor(state.reshape(1, -1)).to(device) 
  return self.actor(state).cpu().data.numpy().flatten() 
  # Need to convert to numpy remember clipping!



Sample from a batch of transitions (s, s', a, r) from the memory

In [0]:
def train(self, replay_buffer, iterations, batch_size=100, discount=0.99, \
          tau = 0.005, policy_noise = 0.2, noise_clip=0.5, policy_freq=2):
  for it in range(iterations):
    # Step 4 We sample from a batch of transitions (s, s', a, r) from memory 
    batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = replay_buffer.sample(batch_size)
    state = torch. Tensor(batch_states).to(device) 
    next_state = torch.Tensor(batch_next_states).to(device) 
    action = torch.Tensor(batch_actions).to(device)
    reward = = torch.Tensor(batch_rewards).to(device)
    done = torch.Tensor (batch_dones).to(device)

Step 5: 
Then from each element of the batch, From the next state s\`, the Actor target plays the next action a\`.

In [0]:
self.actor_target.forward(next_state)

We add Gaussian noise to this next action a' and we clamp it in a range
of values supported by the environment. This is the same as exploration!
Step 6:

In [0]:
noise = torch.Tensor(batch_actions).data.normal_(0, policy_noise).to(device) 
noise = noise.clamp(-noise_clip, noise_clip) 
next_action = (next_action + noise).clamp(-self.max_action, self.max_action)

 STEP 7 
The two Critic targets take each the couple (s', a') as input and return two Q values,
Qt1(s', a') and Qt2(s', a') as outputs
The two Critic Targets each take (s`, a`) as input and return two Q-values as output:

image.png

image.png

In [0]:
target_Q1, target_Q2 = self.critic_target.forward(next_state, next_action)

 STEP 8 Keep the minimum of these two Q-Values. his is not target_Q, we are just being lazy, and want to use the same variable name later on. 
 
Step 8: He reep the chimney of these two values 
this is not target_Q, we are just being lazy, and want to use the same variable name later on. 
We keep the minimum of these two Q-values
It represents the approximated values of the next state.
Taking a minimum of the 2 Q-values prevents too optimistic estimates of that value of the state!

In classic Actor-Critic Method (with 1 Critic) we had overly optimistic estimates which prevented the training process from being stable,

and taking the minimum of 2 Q-Values here adds that stability which was required. 

image.png

In [0]:
target_Q = torch.min(target_Q1, target_Q2)

 STEP 9 
We get the final target of the two Critic models, which is:
Qt = r + gamma * min(Qt1, Qt2)
o
We can define target_q or 
Qt = reward + discount  * Qt

but it won't work...

First, we are only supposed to run this if the episode is over(=1), which means we need to integrate Done.

Second, target_q would create it's BP/computation graph, and without detaching Qt1/Qt2 from their own graph, we are complicating things, i.e. we need to use detach. Let's look below:

In [0]:
target_Q = reward + ((1-done) * discount * target_Q).detach()

 STEP 10 
Two critic models take (s, a) and return two Q-Vales

image.png

image.png

In [0]:
current_Q1, current_Q2 = self.critic.forward(state, action)

 STEP 11 Compute the Critic Loss
 We compute the loss coming from the two Critic models: 

image.png

In [0]:
critic_loss = Mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)

 STEP 12 
Backpropagate this critic loss and update the parameters of two
Critic models with Adam Optimizer

In [0]:
selt.critic_optimizer.zero_grad() #Initialize the gradients to zero
critic_loss.backward() #Computing the gradients
self.critic_optimizer.step() #Performing the weight updates

 STEP 13 Once every two iterations, we update our Actor model by performing
gradient ASCENT on the output of the first Critic model

In [0]:
if it % policy_freq == 0:
  #This is DPG part 
  actor_loss = -(self.critic.Q1(state, self.actor(state)).mean())
  self.actor_optimizer.grad_zero() 
  actor_loss.backward() 
  self.actor_optimizer.step()

 STEP 14 Still, in once every two iterations, we update our Actor Target
by Polyak Averaging
Now, why do we have Different Actor Target and Actor Models? 
Well, they can be the same, and in fact, in many naive RL models, they are the same. 
But we can improve overall performance by keeping two models and updating them from each other. 
Once every two iterations, we update the weights of the Actor target by Polyak averaging

image.png

In [0]:
for param, target_param in zip(self.actor.parameters (), self.actor_target.parameters()):
  target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data) 

This way our target comes closer to the model. 

 STEP 15 Still, in once every two iterations, we update our Critic Target
by Polyak Averaging
Same steps for the Critics
Once every two iterations, we update the weights of the Critic target by Polyak averaging
Where is the DELAYED part of the T3D?
We update our models at every step, but our target once every two steps. 

In [0]:
for param, target_param in zip(self.critical.parameters(), self.critic_target.parameters()):
  target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
